In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from datetime import date, datetime
import time
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
import pickle
import spacy
import random
import ast
import seaborn as sns
import networkx as nx

In [3]:
tqdm.pandas()

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
# getting the burst timeline..
df_tweets = pd.read_csv('/home/auppal8/HexagonData_new.csv',lineterminator='\n', index_col=None)

In [6]:
df_timeline = pd.read_csv('/home/auppal8/userTimelineData.csv',lineterminator='\n',)

In [7]:
df_following = pd.read_excel('/home/auppal8/updatedfollowingData1117.xlsx', index_col=0)

In [8]:
# following data for user...
df_300 = pd.read_csv("~/Following300_1220.csv",index_col=0)

In [9]:
#no of hashtags per tweet
# url and then no of url per tweet
# no of @user-mention per tweet
# percentage of retweet per user (user retweets / total no of tweets)

In [10]:
def hashtag_count(df):
    if df is not np.nan:
        hashtags = ast.literal_eval(df)
        if (hashtags is not None):
            return (len(hashtags))
        else:
            return(0)
    else:
        return(0)

In [11]:
# count no of hashtags
df_timeline['hashtags'] = df_timeline['hashtags'].progress_apply(hashtag_count)

100%|██████████| 1344182/1344182 [00:10<00:00, 128569.39it/s]


In [12]:
# count the no of urls per tweet
df_timeline['urls'] = df_timeline['tweetText'].str.count(r'(https?://\S+)')

In [13]:
# count no of user mentiosn per tweet
df_timeline['author_mentions'] = df_timeline['tweetText'].str.count(r'(\@\w+)')

In [26]:
# just to get avergae user mentions per user
df_user = df_timeline.groupby(df_timeline['userID']).agg({'author_mentions': 'median',
                                                          'urls' : 'median',
                                                          'retweetCount' : 'sum'})

In [33]:
total = df_user['retweetCount'].sum()
df_user['retweetCount'] = df_user['retweetCount'].apply(lambda x: (x/total) * 100)

In [37]:
df_user

7372

In [38]:
# start with behavious characteristics